In [2]:
import util.misc as misc
import torch
from llama.llama_adapter import LLaMA_adapter
import llama

In [3]:
model = llama.whisper_llama_adapter.load("/home/u2619111/hank/lab/big-superb/LLaMA-Adapter/imagebind_LLM/exp/whisper_debug/checkpoint-1.pth", "/home/u2619111/hank/lab/big-superb/LLaMA-Adapter/imagebind_LLM/ckpts/llama", knn=True, max_batch_size=16)
model = model.to("cpu")

Loading LLaMA-Adapter from /home/u2619111/hank/lab/big-superb/LLaMA-Adapter/imagebind_LLM/exp/whisper_debug/checkpoint-1.pth
model args: ModelArgs(dim=4096, n_layers=32, n_heads=32, vocab_size=-1, multiple_of=256, norm_eps=1e-06, max_batch_size=16, max_seq_len=512, w_bias=True, w_lora=True, lora_rank=16)


In [4]:
model2 = llama.whisper_llama_adapter.load("/home/u2619111/hank/lab/big-superb/LLaMA-Adapter/imagebind_LLM/exp/whisper_debug/model-1.pth", "/home/u2619111/hank/lab/big-superb/LLaMA-Adapter/imagebind_LLM/ckpts/llama", knn=True, max_batch_size=16)
model2 = model2.to("cpu")
torch.cuda.empty_cache()

Loading LLaMA-Adapter from /home/u2619111/hank/lab/big-superb/LLaMA-Adapter/imagebind_LLM/exp/whisper_debug/model-1.pth
model args: ModelArgs(dim=4096, n_layers=32, n_heads=32, vocab_size=-1, multiple_of=256, norm_eps=1e-06, max_batch_size=16, max_seq_len=512, w_bias=True, w_lora=True, lora_rank=16)


In [5]:

for name in model.state_dict().keys():
    if not torch.all(model.state_dict()[name] == model2.state_dict()[name]):
        print(
            name
        )

In [4]:
misc.load_model(model, "/home/u2619111/hank/lab/big-superb/LLaMA-Adapter/imagebind_LLM/exp/new_train2/checkpoint-3.pth")

<All keys matched successfully>
Load checkpoint /home/u2619111/hank/lab/big-superb/LLaMA-Adapter/imagebind_LLM/exp/new_train2/checkpoint-3.pth


In [ ]:
checkpoint_paths = [output_dir / ('model-%s.pth' % epoch_name)]
for checkpoint_path in checkpoint_paths:

    model_state_dict = model_without_ddp.state_dict()
    model_state_dict_with_grad = OrderedDict()
    for key, val in model_without_ddp.named_parameters():
        if val.requires_grad:
            model_state_dict_with_grad[key] = model_state_dict[key]

    to_save = {
        'model': model_state_dict_with_grad,
        # 'optimizer': optimizer.state_dict(),
        # 'epoch': epoch,
        # 'scaler': loss_scaler.state_dict(),
        'args': args,
    }

    save_on_master(to_save, checkpoint_path)

In [9]:
from collections import defaultdict, deque, OrderedDict

model_state_dict = model.state_dict()
model_state_dict_with_grad = OrderedDict()
for key, val in model.named_parameters():
    if val.requires_grad:
        model_state_dict_with_grad[key] = model_state_dict[key]
to_save = {
        'model': model_state_dict_with_grad,
        # 'optimizer': optimizer.state_dict(),
        # 'epoch': epoch,
        # 'scaler': loss_scaler.state_dict(),
        # 'args': args,
    }
torch.save(to_save, "/home/u2619111/hank/lab/big-superb/LLaMA-Adapter/imagebind_LLM/exp/new_train2/model-3.pth")

In [1]:
# Hank
import torch
import yaml
from torch.utils.data import Dataset
from PIL import Image
import json
import llama.utils
from llama import Tokenizer
import copy
import torchvision.transforms as transforms
import pandas as pd
import random
from pytorchvideo.data.clip_sampling import ConstantClipsPerVideoSampler
import torchaudio
import whisper

try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC

class BigSuperbDataset(Dataset):
    def __init__(self, data_path, tokenizer, data_path2=None, max_length=128, used_data_split="train", phase="train", audio_input_type="imagebind", allowed_datasets=[]):
        self.max_length = max_length
        self.tokenizer = tokenizer
        self.datas = []
        self.used_datasets = []
        self.used_data_split = used_data_split
        self.audio_input_type = audio_input_type
        self.allowed_datasets = allowed_datasets
        self.phase = phase

        for task_path in data_path.iterdir():
            if not self.check_allowed_datasets(task_path):
                continue
            
            self.used_datasets.append(task_path.stem)
            
            for data_split in task_path.iterdir():
                if data_split.stem != self.used_data_split:
                    continue
                
                json_data = json.load((data_split/"metadata.json").open())
                for filename, d in json_data.items():
                    
                    d["file"] = f"{data_split}/{filename}"

                    if d.get("file2"):
                        filename2 = filename.replace(".wav", "_pair.wav").replace(".flac", "_pair.flac")
                        d["file2"] = f"{data_split}/{filename2}"
                        # assert Path(d["file2"]).exists(), d["file2"]
                        # if "." not in d["file2"]:
                        #     d["file2"] = d["file2"] + ".wav"
                        
                        # if (data_split/f"{task_path.stem}_{self.used_data_split}_{d['file2']}").exists():
                        #     d["file2"] = f"{data_split}/{task_path.stem}_{self.used_data_split}_{d['file2']}"
                        # elif (task_path/"missing_files"/d["file2"]).exists():
                        #     d["file2"] = str(task_path/"missing_files"/d["file2"])
                        # else:
                        #     assert False, f"{task_path}"+ d["file2"]
                            
                    self.datas.append(d)
        # exclude
        if data_path2 is not None:
            for task_path in data_path2.iterdir():
                if not self.check_allowed_datasets(task_path):
                    continue
                
                self.used_datasets.append(task_path.stem)
                for data_split in task_path.iterdir():
                    if data_split.stem != self.used_data_split:
                        continue
                    
                    json_data = json.load((data_split/"metadata.json").open())
                    for file_name, d in json_data.items():
                        d["file"] = str(data_split/file_name)
                                
                        self.datas.append(d)


        # Audio loader
        self.clip_sampler = ConstantClipsPerVideoSampler(
            clip_duration=2, clips_per_video=3
        )
        print("Used datasets", len(self.used_datasets), self.used_datasets)
        print(len(self.datas))

    def __len__(self):
        return len(self.datas)

    def __getitem__(self, idx):
        data = self.datas[idx]
        new_data = {}
        
        # Text 
        if self.used_data_split == "train":
            instruction = data["instruction"].lower()
            text = data["text"].lower() if data.get("text") else None
            input1 = llama.format_prompt(instruction, text)
            input2 = input1 + data["label"]

            input1 = torch.tensor(
                self.tokenizer.encode(input1, bos=True, eos=False), dtype=torch.long
            )
            input2 = torch.tensor(self.tokenizer.encode(input2, bos=True, eos=True), dtype=torch.long)
            padding = self.max_length - input2.size(0)
            if padding > 0:
                input2 = torch.cat((input2, torch.zeros(padding, dtype=torch.long) - 1))
            else:
                input2 = input2[:self.max_length]

            labels = copy.deepcopy(input2)
            labels[:input1.size(0)] = -1

            input2_mask = input2.ge(0)
            label_mask = labels.ge(0)
            input2[~input2_mask] = 0
            labels[~label_mask] = 0

            input2_mask = input2_mask.float()
            label_mask = label_mask.float()

            new_data["instruction"] = data["instruction"]
            new_data["input_ids"] = input2
            new_data["labels"] = labels
            new_data["input_mask"] = input2_mask

            # for inference training set
            if self.phase == "inference":
                instruction = data["instruction"].lower()
                text = data["text"].lower() if data.get("text") else None
                input1 = llama.format_prompt(instruction, text)

                new_data["prompt"] = input1
                new_data["instruction"] = data["instruction"]
                new_data["label"] = data["label"]
        elif self.used_data_split == "test":
            instruction = data["instruction"].lower()
            text = data["text"].lower() if data.get("text") else None
            input1 = llama.format_prompt(instruction, text)
            
            new_data["prompt"] = input1
            new_data["instruction"] = data["instruction"]
            new_data["label"] = data["label"]

        # Load Audio
        if self.audio_input_type == "imagebind":
            if data.get("file2"):
                audio = self._load_imagebind_audio([data["file"], data["file2"]])
            else:
                audio = self._load_imagebind_audio([data["file"]])
        elif self.audio_input_type == "whisper":
            if data.get("file2"):
                audio = self._load_whisper_audio([data["file"], data["file2"]])
            else:
                audio = self._load_whisper_audio([data["file"]])
        new_data["audio"] = audio
        return new_data
    
    def _load_whisper_audio(self, audio_paths):

        waveforms = []
        for audio_path in audio_paths:
            waveform = torch.tensor(whisper.load_audio(audio_path))
            if waveform.size(0) == 0:
                waveform = torch.zeros([16000*3])
                print(audio_path)
            
            if len(audio_paths) == 2:
                # pad_or_trim to 15 seconds for tasks have 2 audio.
                waveform = whisper.pad_or_trim(waveform, length=15*16000)
            waveforms.append(
                waveform
            )
        audio = torch.cat(waveforms, dim=0)
        audio = whisper.pad_or_trim(audio)
        mel = whisper.log_mel_spectrogram(audio)
        
        return mel

    
    def _load_imagebind_audio(self, 
            audio_paths,
            num_mel_bins=128,
            target_length=204,
            sample_rate=16000,
            clip_duration=2,
            clips_per_video=3,
            mean=-4.268,
            std=9.138
        ):

        waveforms = []
        for audio_path in audio_paths:
            waveform, sr = torchaudio.load(audio_path)

            if waveform.size(1) == 0:
                waveform = torch.zeros([1, 16000*3])
                sr = 16000
                # logging.warning(f"An audio is set to zero, {audio_path}")
                print(audio_path)
                
            if sample_rate != sr:
                waveform = torchaudio.functional.resample(
                    waveform, orig_freq=sr, new_freq=sample_rate
                )

            if len(audio_paths) == 2:
                if waveform.size(1) < 5*16000:
                    waveform = torch.cat(
                        [waveform, torch.zeros([1, 5*16000 - waveform.size(1)])], dim=1
                    )
                else:
                    waveform = waveform[:, :5*16000]

            waveforms.append(waveform)
        waveform = torch.cat(waveforms, dim=1)
        
            
        all_clips_timepoints = self._get_clip_timepoints(
            self.clip_sampler, waveform.size(1) / sample_rate
        )
        all_clips = []
        for clip_timepoints in all_clips_timepoints:
            waveform_clip = waveform[
                :,
                int(clip_timepoints[0] * sample_rate) : int(
                    clip_timepoints[1] * sample_rate
                ),
            ]
            waveform_melspec = self._def_waveform2melspec(
                waveform_clip, sample_rate, num_mel_bins, target_length
            )
            all_clips.append(waveform_melspec)

        normalize = transforms.Normalize(mean=mean, std=std)
        all_clips = [normalize(ac) for ac in all_clips]

        all_clips = torch.stack(all_clips, dim=0)

        return all_clips
        
    def _get_clip_timepoints(self, clip_sampler, duration):
        # Read out all clips in this video
        all_clips_timepoints = []
        is_last_clip = False
        end = 0.0
        while not is_last_clip:
            start, end, _, _, is_last_clip = clip_sampler(end, duration, annotation=None)
            all_clips_timepoints.append((start, end))
        return all_clips_timepoints

    def _def_waveform2melspec(self, waveform, sample_rate, num_mel_bins, target_length):
        # Based on https://github.com/YuanGongND/ast/blob/d7d8b4b8e06cdaeb6c843cdb38794c1c7692234c/src/dataloader.py#L102
        waveform -= waveform.mean()
        fbank = torchaudio.compliance.kaldi.fbank(
            waveform,
            htk_compat=True,
            sample_frequency=sample_rate,
            use_energy=False,
            window_type="hanning",
            num_mel_bins=num_mel_bins,
            dither=0.0,
            frame_length=25,
            frame_shift=10,
        )
        # Convert to [mel_bins, num_frames] shape
        fbank = fbank.transpose(0, 1)
        # Pad to target_length
        n_frames = fbank.size(1)
        p = target_length - n_frames
        
        # cut and pad
        if p > 0:
            fbank = torch.nn.functional.pad(fbank, (0, p), mode="constant", value=0)
        elif p < 0:
            fbank = fbank[:, 0:target_length]
        # Convert to [1, mel_bins, num_frames] shape, essentially like a 1
        # channel image
        fbank = fbank.unsqueeze(0)
        return fbank

    def check_allowed_datasets(self, task_path):
        for d in self.allowed_datasets:
            if task_path.stem.lower() in d.lower():
                return True
        # print(f"find {task_path.stem} not in allowed list.")
        return False

/home/u2619111/.local/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
/home/u2619111/.local/lib/python3.9/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):


In [2]:
from pathlib import Path
tokenizer = Tokenizer(model_path="/home/u2619111/hank/lab/big-superb/LLaMA-Adapter/imagebind_LLM/ckpts/llama/tokenizer.model")
data_path = Path("/home/u2619111/hank/Dataset/big-superb-test-data-renamed")
all_datasets = open("data/test_dataset.txt").read().split("\n")
print(all_datasets)
# all_datasets = ["SpeakerVerification_LibrispeechTrainClean100"]
all_train_dataset = BigSuperbDataset(data_path, tokenizer, data_path2=None, used_data_split="test", audio_input_type="whisper", phase="test", allowed_datasets=all_datasets)

['BirdSoundDetection_Warblrb10k', 'ChordClassification_AcousticGuitarAndPiano', 'EnvironmentalSoundClassification_AnimalsESC50', 'EnvironmentalSoundClassification_ExteriorAndUrbanNoisesESC50', 'EnvironmentalSoundClassification_HumanAndNonSpeechSoundsESC50', 'EnvironmentalSoundClassification_InteriorAndDomesticSoundsESC50', 'EnvironmentalSoundClassification_NaturalSoundscapesAndWaterSoundsESC50', 'SpeechDetection_LJSpeech', 'SpeechDetection_LibriSpeechTestClean', 'SpeechDetection_LibriSpeechTestOther', 'SpeechTextMatching_LJSpeech', 'SpeechTextMatching_LibriSpeechTestClean', 'SpeechTextMatching_LibriSpeechTestOther', 'SpokenTermDetection_LJSpeech', 'SpokenTermDetection_LibriSpeechTestClean', 'SpokenTermDetection_LibriSpeechTestOther', 'SpeechCommandRecognition_GoogleSpeechCommandsV1', 'LanguageIdentification_VoxForge', 'EnhancementDetection_LibrittsTestCleanWham', 'NoiseDetectiongaussian_LJSpeechMusan', 'NoiseDetectiongaussian_VCTKMusan', 'NoiseDetectionmusic_LJSpeechMusan', 'NoiseDetec

In [3]:
for i in range(0, len(all_train_dataset), 500):
    all_train_dataset[i]

In [3]:
from pathlib import Path
import random
import json
from collections import Counter
for task_path in Path("/home/u2619111/hank/Dataset/big-superb-test-data-renamed").iterdir():
    datas = json.load((task_path/"test/metadata.json").open())
    
    all_labels = []
    for file_name, data in datas.items():
        all_labels.append(data["label"])
    random.seed(1) # 42, 1 ,87
    predictions = []
    for file_name, data in datas.items():
        predictions.append({
            "pred": random.choice(all_labels),
            "label": data["label"],
            "instruction": data["instruction"]
        })

    counter = Counter(all_labels)
    json.dump({"label_count": counter,"predictions": predictions},
        open(f"/home/u2619111/hank/lab/big-superb/LLaMA-Adapter/imagebind_LLM/exp/random/random1/{task_path.stem}.json", "w"), indent=2, ensure_ascii=False)
